In [2]:
import torch
import gradio as gr
import os
from transformers import pipeline

# Make sure the HUGGINGFACE_TOKEN environment variable is set
token = os.getenv('HUGGINGFACE_TOKEN')
device = 0 if torch.cuda.is_available() else -1

# Create the pipeline for text generation using the LLaMA model
pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B-Instruct", device=device, token=token)

def llama_inference(user_input):
    messages = [{"role": "user", "content": user_input}]
    response = pipe(messages, max_length=100)
    return response[0]['generated_text'][1]['content']

# Create the Gradio interface
iface = gr.Interface(
    fn=llama_inference,
    inputs="text",
    outputs="text",
    title="LLaMA Text Generation",
    description="Enter a prompt and get a response generated by the LLaMA model."
)

# Launch the Gradio app
iface.launch()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [10]:
iface.close()

Closing server running on port: 7861
